#### usefull source:
https://www.geeksforgeeks.org/python-removing-unwanted-characters-from-string/

In [1]:
import os
import pandas as pd
import numpy as np

# Extract data from gamePass

In [2]:
dir_path = "./Game-Pass-API/"
if os.path.isdir(dir_path):
    print(f"{dir_path} exists.")
else:
    print(f"{dir_path} does not exist.")
    #creation of dir:
    os.system('git clone https://github.com/NikkelM/Game-Pass-API.git')

./Game-Pass-API/ exists.


In [3]:
os.chdir("./Game-Pass-API/")#we change current path so let's not forget to come back to the root of the project
os.system("npm install")
os.chdir("..")

In [4]:
os.system('node Game-Pass-API/index.js')

1

In [5]:
gamepass_games=pd.read_json("Game-Pass-API/output/formattedGameProperties_console_US.json")
gamepass_games

,productTitle
0,7 Days to Die
1,A Little to the Left
2,A Plague Tale: Requiem
3,A Short Hike
4,A Way Out
...,...
500,You Suck at Parking® Complete Edition
501,Zombie Army 4: Dead War
502,Zoo Tycoon: Ultimate Animal Collection
503,Zuma


# transformation: need to lowercase and remove special characters

In [6]:
gamepass_games["productTitle"][25]="Banjo-Kazooie: Nuts & Bolts"

In [7]:
gamepass_games["productTitle"][25]

'Banjo-Kazooie: Nuts & Bolts'

In [8]:
import unidecode

list_transformed_title=[]
# initializing bad_chars_list
# r copyright -> spe@#$ci87al*& ou ®
bad_chars = ['.',';', ':', '!','\'', "*", "®","™"]
# using replace() to
# remove bad_chars

for title in gamepass_games['productTitle']:
    title=title.replace(" for Xbox","").replace(" - Xbox One Edition","").replace(" - Xbox Series X|S","").replace(" for Xbox One","").replace(" Preview","").replace(" (Game Preview)","").replace(" (Game Preview)","").replace(" Standard Edition","").replace(" Console","").replace(" (Xbox One)","").replace(" Xbox Series X|S","").replace(" Xbox One","")
    title=title.replace(" - ","-").replace(": ","-").replace("&","and").replace(" ","-")
    title = ''.join((filter(lambda i: i not in bad_chars, title)))
    title=title.lower()
    list_transformed_title.append(unidecode.unidecode(title))
    

In [9]:
gamepass_games["tr_productTitle"]=pd.DataFrame(list_transformed_title)

# Extract data from Metacritic

In [12]:
from bs4 import BeautifulSoup
import requests, random


AGENTS = ('Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
          'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36')

#create headers
HEADERS               = requests.utils.default_headers()
HEADERS['User-Agent'] = random.choice(AGENTS)

### 1st passage to get all url with problems to correct

In [14]:
list_Of_Score=[]
list_Of_NotGood=[]#list of url not well formated for metacritic,needed to be reworked
it=0
for tr_title in gamepass_games["tr_productTitle"]:
    it+=1
    url = "https://www.metacritic.com/game/"+tr_title+"/"
    #print(url,it-1)
    page = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(page.content, 'html.parser')
    score_text = soup.find("span", {"class": "c-productScoreInfo_scoreSentiment"})
    if score_text is None:
        #print("not good")
        list_Of_NotGood.append(it-1)
        list_Of_Score.append("unknown")
    else:
        list_Of_Score.append(score_text.get_text().replace("\n ","").replace("\n",""))
gamepass_games["metascore"]=pd.DataFrame(list_Of_Score)

In [15]:
gamepass_games.head(3)

,productTitle,tr_productTitle,metascore
0,7 Days to Die,7-days-to-die,Generally Unfavorable
1,A Little to the Left,a-little-to-the-left,Generally Favorable
2,A Plague Tale: Requiem,a-plague-tale-requiem,Generally Favorable


In [16]:
gamepass_games["metascore"][gamepass_games["metascore"]!="not good"].count()

443

In [17]:
gamepass_games["metascore"][gamepass_games["metascore"]=="not good"].count()

62